In [1]:
import json
import pandas as pd
import numpy as np

In [23]:
import os
data_path = './카카오 아레나 멜론/'
data_list = os.listdir(data_path)
data_list

['genre_gn_all.json', 'song_meta.json', 'test.json', 'train.json', 'val.json']

# genre_gn_all
장르에 대한 정보를 담은 json 파일

||장르 코드|장르|
|---|---|---|
|설명|장르 코드|한글로 쓴 장르|
|예시|GN0100|(00으로 끝나는 대분류) 발라드|
|예시|GN2902|(00으로 안끝나는 분류) 국내뮤지컬|

254 rows × 2 columns

In [51]:
genre_gn_all = pd.read_json(data_path + data_list[0], orient = 'index')
genre_gn_all = pd.DataFrame(genre_gn_all) 
genre_gn_all.head() 

,0
GN0100,발라드
GN0101,세부장르전체
GN0102,'80
GN0103,'90
GN0104,'00


In [52]:
genre_gn_all.reset_index(inplace = True)
genre_gn_all.columns = ['장르 코드', '장르']
genre_gn_all

,장르 코드,장르
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
...,...,...
249,GN2900,뮤지컬
250,GN2901,세부장르전체
251,GN2902,국내뮤지컬
252,GN2903,국외뮤지컬


## 장르 대분류, 소분류로 분리 - genre_code
장르 코드가 00으로 끝나면 대분류, 00이 아니라 다른 숫자로 끝나면 소분류

In [74]:
big_cat = genre_gn_all[genre_gn_all['장르 코드'].str[-2:] == '00'] 
big_cat.head() 

,장르 코드,장르
0,GN0100,발라드
6,GN0200,댄스
12,GN0300,랩/힙합
18,GN0400,R&B/Soul
22,GN0500,인디음악


In [75]:
sml_cat = genre_gn_all[genre_gn_all['장르 코드'].str[-2:] != '00']
sml_cat.head() 

,장르 코드,장르
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
5,GN0105,'10-


In [76]:
# 앞자리 네 자리 공통코드 추출
big_cat = big_cat.assign(join_code = big_cat['장르 코드'].str[0:4])
sml_cat = sml_cat.assign(join_code = sml_cat['장르 코드'].str[0:4])

# Merge
genre_code = pd.merge(big_cat, sml_cat, how = 'left', on = 'join_code')

In [77]:
genre_code.drop('join_code', inplace = True, axis = 1)
genre_code.columns = ['장르 대분류', '대분류 설명', '장르 소분류', '소분류 설명']
genre_code

,장르 대분류,대분류 설명,장르 소분류,소분류 설명
0,GN0100,발라드,GN0101,세부장르전체
1,GN0100,발라드,GN0102,'80
2,GN0100,발라드,GN0103,'90
3,GN0100,발라드,GN0104,'00
4,GN0100,발라드,GN0105,'10-
...,...,...,...,...
220,GN2800,뮤직테라피,GN2806,반려동물
221,GN2900,뮤지컬,GN2901,세부장르전체
222,GN2900,뮤지컬,GN2902,국내뮤지컬
223,GN2900,뮤지컬,GN2903,국외뮤지컬


In [84]:
genre_code.to_excel(data_path + '장르 코드.xlsx', index = False)

# song_meta
노래별 메타 데이터

||song_gn_dtl_gnr_basket|issue_date(yyyymmdd)|album_name|album_id|artist_id_basket|song_name|song_gn_gnr_basket|artist_name_basket|id|
|---|---|---|---|---|---|---|---|---|---|
|설명|세부 장르|발매일|앨범 이름|앨범 ID|아티스트 ID|곡 제목|장르|아티스트|곡 ID|
|예시|[GN1601, GN1606]|20080421|Bach : Partitas Nos. 2, 3 & 4|376431|[29966]|Bach : Partita No.4 In D Major, BWV 828 -...|[GN1600]|[Murray Perahia]|1|
|예시|[GN1801]|20121219|멤돌다|2170485|[714002]|말하지 못한 이야기|[GN1800]|[페르마타]|422|

707989 rows × 9 columns

In [57]:
song_meta = pd.read_json(data_path + data_list[1])
song_meta = pd.DataFrame(song_meta) 
song_meta.head() 

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [78]:
song = song_meta.copy()
song.columns = ['세부 장르', '발매일', '앨범 이름', '앨범 ID', '아티스트 ID', '곡 제목', '장르', '아티스트', '곡 ID']
song

,세부 장르,발매일,앨범 이름,앨범 ID,아티스트 ID,곡 제목,장르,아티스트,곡 ID
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [64]:
song.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   세부 장르    707989 non-null  object
 1   발매일      707989 non-null  int64 
 2   앨범 이름    707985 non-null  object
 3   앨범 ID    707989 non-null  int64 
 4   아티스트 ID  707989 non-null  object
 5   곡 제목     707989 non-null  object
 6   장르       707989 non-null  object
 7   아티스트     707989 non-null  object
 8   곡 ID     707989 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 48.6+ MB


# train
모델 학습에 쓸 train set

||tags|id|plylst_title|songs|like_cnt|updt_date|
|---|---|---|---|---|---|---|
|설명|태그|플리 ID|플리 제목|곡 리스트|좋아요 개수|수정 날짜|
|예시|[추억, 회상]|10532|요즘 너 말야|[432406, 675945, 497066, 120377, 389529, 24427...]|1|2014-12-02 16:19:42.000|
|예시|[노래추천, 팝송추천, 팝송, 팝송모음]|100389|FAVORITE POPSONG!!!|[26008, 456354, 324105, 89871, 135272, 143548,...]|17|2020-04-18 20:35:06.000|

115071 rows × 6 columns

In [80]:
data_path + data_list[3] 

'./카카오 아레나 멜론/train.json'

In [81]:
train = pd.read_json(data_path + data_list[3]) 
train = pd.DataFrame(train)
train.head() 

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
